In [2]:
import numpy as np
import tensorflow as tf

a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)  # 类型推导dtype=tf.float32
c = tf.constant(4)
total = a + b
print(a)
print(b)
print(c)
print(total)  # 张量是根据生成它们的指令命名的，后面跟着输出索引

Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Const_3:0", shape=(), dtype=float32)
Tensor("Const_4:0", shape=(), dtype=int32)
Tensor("add_1:0", shape=(), dtype=float32)


In [3]:
writer = tf.summary.FileWriter('.')  # 将在当前目录中生成一个 event 文件
writer.add_graph(tf.get_default_graph())
# 名称格式:events.out.tfevents.{timestamp}.{hostname}

In [4]:
# 新的终端中使用以下 shell 命令启动 TensorBoard
!tensorboard --logdir .  

Traceback (most recent call last):
  File "c:\users\lxg\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\lxg\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\lxg\AppData\Local\Programs\Python\Python36\Scripts\tensorboard.exe\__main__.py", line 9, in <module>
  File "c:\users\lxg\appdata\local\programs\python\python36\lib\site-packages\tensorboard\main.py", line 58, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "c:\users\lxg\appdata\local\programs\python\python36\lib\site-packages\absl\app.py", line 300, in run
    _run_main(main, args)
  File "c:\users\lxg\appdata\local\programs\python\python36\lib\site-packages\absl\app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "c:\users\lxg\appdata\local\programs\python\python36\lib\site-packages\tensorboard\program.py", line 194, in main
    s

In [6]:
# 会话
sess = tf.Session()
print(sess.run(total))

7.0


In [7]:
# 可以将多个张量传递给 tf.Session.run。run 方法以透明方式处理元组或字典的任何组合
print(sess.run({'ab': (a, b), 'total': total}))

{'ab': (3.0, 4.0), 'total': 7.0}


In [9]:
vec = tf.random_uniform(shape=(3,))  # 值位于 [0,1) 区间内
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.40174532 0.61116004 0.9165559 ]
(array([1.098931 , 1.0107367, 1.8000283], dtype=float32), array([2.0989308, 2.0107367, 2.8000283], dtype=float32))


In [10]:
# 占位符表示承诺在稍后提供值，它就像函数参数
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(sess.run(z, feed_dict={x: 3, y: 4.5}))  # feed_dict 参数为占位符提供具体的值
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


In [12]:
# 要从数据集中获取可运行的 tf.Tensor，您必须先将其转换成 tf.data.Iterator，然后调用迭代器的 get_next 方法
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


In [13]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
# 如果 Dataset 依赖于有状态操作，则可能需要在使用迭代器之前先初始化它
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        break

[ 0.9761834  -0.08618497 -1.6243111 ]
[-2.0997748   1.6534332  -0.94701344]
[-1.6254568  -0.7428545   0.43614793]
[-3.1429133   0.01579424  0.20279098]
[1.9987543 1.774132  1.6975774]
[-2.2472281  -0.15737073  0.6753185 ]
[-0.3184826 -0.3052357 -0.8068632]
[-1.7214998  -0.43616045 -1.4863175 ]
[ 1.3718286  -1.2475213  -0.73278815]
[-0.9447189   0.08658594  0.81219304]


In [15]:
# 层将变量和作用于它们的操作打包在一起
# 创建层
x = tf.placeholder(tf.float32, shape=(None, 3))
linear_model = tf.layers.Dense(units=1)  # 层
y = linear_model(x)

In [16]:
# 层包含的变量必须先初始化，然后才能使用
# 初始化层
init = tf.global_variables_initializer()
sess.run(init)

In [17]:
# 执行层
print(sess.run(y, {x:[[1, 2, 3], [4, 5, 6]]}))

[[ 0.4081741 ]
 [-0.00965703]]


In [20]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)  # 层函数的快捷方式是在单次调用中创建和运行层

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1,2,3],[4,5,6]]}))

[[-0.85936964]
 [-0.3459444 ]]


In [26]:
# 特征列
features = {
    'sales': [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']
}
department_column = tf.feature_column.categorical_column_with_vocabulary_list(
    'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [27]:
# 特征列和层一样具有内部状态，因此通常需要将它们初始化
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

print(sess.run(inputs))  # 将department列 one-hot编码

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


In [29]:
x = tf.constant([[1],[2],[3],[4]], dtype=tf.float32)
y_true = tf.constant([[0],[-1],[-2],[-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[0.45253623]
 [0.90507245]
 [1.3576087 ]
 [1.8101449 ]]


In [31]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
print(sess.run(loss))  # 损失值

9.56128


In [34]:
# 优化器
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

9.56128
9.254329
8.957539
8.670574
8.393107
8.124826
7.865424
7.6146092
7.3720956
7.137608
6.910883
6.6916614
6.4796944
6.2747436
6.076574
5.8849635
5.6996937
5.5205536
5.3473425
5.179863
5.017924
4.8613434
4.709944
4.563553
4.4220047
4.285139
4.1528015
4.0248413
3.9011137
3.7814784
3.6658003
3.553948
3.4457946
3.3412175
3.240099
3.1423235
3.047781
2.9563644
2.86797
2.7824981
2.699851
2.619936
2.5426617
2.4679418
2.3956912
2.3258276
2.2582724
2.1929495
2.129784
2.0687056
2.009644
1.9525334
1.8973086
1.8439075
1.7922695
1.742336
1.694051
1.6473596
1.6022089
1.5585482
1.5163281
1.4755006
1.43602
1.3978413
1.3609216
1.3252193
1.2906935
1.2573062
1.2250189
1.1937954
1.1636004
1.1344001
1.1061614
1.0788523
1.0524423
1.0269014
1.0022008
0.9783129
0.9552106
0.9328679
0.9112598
0.8903617
0.8701504
0.850603
0.83169734
0.8134123
0.7957275
0.77862287
0.76207924
0.7460781
0.7306013
0.71563184
0.70115244
0.68714726
0.6736003
0.6604966
0.6478214
0.63556063
0.6237004
0.61222744


In [2]:
# 完整程序
import tensorflow as tf

x = tf.constant([[1],[2],[3],[4]], dtype=tf.float32)
y_true = tf.constant([[0],[-1],[-2],[-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

print(sess.run(y_pred))

0.22121112
0.20604885
0.19521381
0.1873833
0.18163931
0.17734516
0.1740587
0.17147338
0.16937637
0.16762003
0.16610183
0.16475067
0.16351718
0.16236717
0.16127677
0.1602295
0.15921393
0.15822205
0.15724833
0.15628895
0.15534116
0.15440314
0.1534735
0.1525515
0.15163629
0.15072754
0.14982489
0.14892805
0.14803691
0.14715132
0.1462712
0.14539644
0.14452696
0.14366272
0.1428037
0.14194986
0.14110112
0.14025748
0.13941887
0.13858528
0.13775672
0.13693307
0.13611437
0.13530058
0.13449164
0.13368753
0.13288821
0.13209371
0.13130397
0.1305189
0.12973855
0.12896287
0.1281918
0.12742536
0.12666352
0.12590623
0.12515344
0.124405175
0.12366138
0.122922026
0.12218711
0.12145655
0.1207304
0.12000854
0.11929101
0.118577816
0.11786887
0.117164135
0.11646365
0.11576732
0.115075186
0.114387155
0.11370322
0.113023445
0.11234769
0.11167598
0.111008264
0.11034458
0.10968483
0.10902905
0.10837717
0.10772921
0.10708512
0.10644485
0.10580845
0.105175845
0.104546994
0.103921965
0.103300616
0.102683015
0.10206